In [10]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *

In [11]:
def procesando_archivo_cancion(cur,filepath):
    # Abrimos el o los archivos JSON
    df = pd.read_json(filepath,lines=True)
    # Insertamos campos de la canción
    song_data = df[["song_id", "title", "artist_id", "year", "duration"]].values[0].tolist()
    cur.execute(song_table_insert,song_data)
    #Procedemos a insertar los artistas
    artist_data = df[["artist_id","artist_name","artist_location","artist_latitude","artist_longitude"]].values[0].tolist()
    cur.execute(artist_table_insert,artist_data)
    
    
def procesando_log_data(cur,filepath):
    #Leemos nuestro archivo JSON
    df = pd.read_json(filepath, lines=True)
    
    #Filtramos por la acción de NextSong
    df = df.query("page == 'NextSong'")
    
    #Convertimos de timestamp a date_time
    t = pd.to_datetime(df['ts'],unit = 'ms')
    df['ts'] = pd.to_datetime(df['ts'],unit ='ms')
    
    #Insertamos los records
    time_data = list((t, t.dt.hour, t.dt.day, t.dt.weekofyear, t.dt.month, t.dt.year, t.dt.weekday))
    column_labels = list(('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday'))
    time_df =  pd.DataFrame.from_dict(dict(zip(column_labels, time_data)))
    
    for i, row in time_df.iterrows():
        cur.execute(time_table_insert,list(row))
    
    #Cargamos la tabla de usuarios
    user_df = df[["userId", "firstName", "lastName", "gender", "level"]]
    
    #Insertamos los campos de usuario
    for i,row in user_df.iterrows():
        cur.execute(user_table_insert,row)
    
    #Insetamos los compas de songplay
    for index, row in df.iterrows():
        #obtenemos el id de la cancion y artista de cancion y artistas tablas
        cur.execute(song_select, (row.song, row.artist, row.length))
        results = cur.fetchone()
        
        if results:
            songid,artistid = results
        else:
            songid,artistid = None, None
            
        #Insertamos los campos de songplay
        songplay_data = (index, row.ts, row.userId, row.level, songid, artistid, row.sessionId,\
                     row.location, row.userAgent)
        
        cur.execute(songplay_table_insert, songplay_data)

        
def process_data(cur, conn, filepath, func):
    # get all files matching extension from directory
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))

    # get total number of files found
    num_files = len(all_files)
    print('{} files found in {}'.format(num_files, filepath))

    # iterate over files and process
    for i, datafile in enumerate(all_files, 1):
        func(cur, datafile)
        conn.commit()
        print('{}/{} files processed.'.format(i, num_files))


        
        
        
def main():
    conn = psycopg2.connect("host=database-1.c295zanvccq2.us-east-1.rds.amazonaws.com dbname=sparkifydb user=student password=root2021")
    cur = conn.cursor()
    
    process_data(cur,conn, filepath='song_data',func=procesando_archivo_cancion)
    process_data(cur,conn, filepath='log_data', func=procesando_log_data)
    conn.close()

In [12]:
if __name__ == "__main__":
    main()

73 files found in song_data
1/73 files processed.
2/73 files processed.
3/73 files processed.
4/73 files processed.
5/73 files processed.
6/73 files processed.
7/73 files processed.
8/73 files processed.
9/73 files processed.
10/73 files processed.
11/73 files processed.
12/73 files processed.
13/73 files processed.
14/73 files processed.
15/73 files processed.
16/73 files processed.
17/73 files processed.
18/73 files processed.
19/73 files processed.
20/73 files processed.
21/73 files processed.
22/73 files processed.
23/73 files processed.
24/73 files processed.
25/73 files processed.
26/73 files processed.
27/73 files processed.
28/73 files processed.
29/73 files processed.
30/73 files processed.
31/73 files processed.
32/73 files processed.
33/73 files processed.
34/73 files processed.
35/73 files processed.
36/73 files processed.
37/73 files processed.
38/73 files processed.
39/73 files processed.
40/73 files processed.
41/73 files processed.
42/73 files processed.
43/73 files pro

c:\users\felipe\documents\curso\udacity\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\felipe\documents\curso\udacity\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


UndefinedColumn: column ss.tittle does not exist
LINE 4: WHERE ss.tittle = 'You Gotta Be'
              ^
HINT:  Perhaps you meant to reference the column "ss.title".
